<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial9_Advanced_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Tutorial9: 発展的なモデルの学習

#### Setup

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  13269      0 --:--:-- --:--:-- --:--:-- 13269


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

     |████████████████████████████████| 532kB 11.6MB/s 


'2.5.0.dev'

#### ハイパーパラメータの最適化

In [3]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', split='scaffold')
train_dataset, valid_dataset, test_dataset = datasets

'split' is deprecated.  Use 'splitter' instead.


- DeepChemでのハイパーパラメータ最適化
    - dc.hyperパッケージで提供。

In [6]:
# 探索パラメータグリッドをせてい
params_dict = {
    'n_tasks': [len(tasks)],
    'n_features': [1024],
    'layer_sizes': [[500],[1000],[1000,1000]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001]
}

optimizer = dc.hyper.GridHyperparamOpt(dc.models.MultitaskClassifier)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
                                                                        params_dict,
                                                                        train_dataset,
                                                                        valid_dataset,
                                                                        metric, transformers
                                                                        )

In [7]:
all_results

{'_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7668880437977661,
 '_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7692549113266706,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7774608686067018,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7829042046345287,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7783243312757202,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7721285273368608,
 '_dropouts_0.500000_layer_sizes[1000, 1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7594843106995885,
 '_dropouts_0.500000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7148031795022536,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.762145

In [10]:
best_hyperparams

(1, 1024, [1000], 0.2, 0.001)

In [11]:
all_results

{'_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7668880437977661,
 '_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7692549113266706,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7774608686067018,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7829042046345287,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7783243312757202,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7721285273368608,
 '_dropouts_0.500000_layer_sizes[1000, 1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7594843106995885,
 '_dropouts_0.500000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7148031795022536,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.762145

In [13]:
# Early Stopping
model = dc.models.MultitaskClassifier(
                                        n_tasks = len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts = 0.2,
                                      learning_rate = 0.0001
                                    )

callback = dc.models.ValidationCallback(valid_dataset, 1000, metric)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

Step 1000 validation: roc_auc_score=0.736116
Step 2000 validation: roc_auc_score=0.763247
Step 3000 validation: roc_auc_score=0.756104
Step 4000 validation: roc_auc_score=0.766018
Step 5000 validation: roc_auc_score=0.775994
Step 6000 validation: roc_auc_score=0.76735
Step 7000 validation: roc_auc_score=0.764298
Step 8000 validation: roc_auc_score=0.767075
Step 9000 validation: roc_auc_score=0.770679
Step 10000 validation: roc_auc_score=0.764105
Step 11000 validation: roc_auc_score=0.767219
Step 12000 validation: roc_auc_score=0.762932
Step 13000 validation: roc_auc_score=0.769099
Step 14000 validation: roc_auc_score=0.767849
Step 15000 validation: roc_auc_score=0.763817
Step 16000 validation: roc_auc_score=0.768698


0.6993769073486328

In [14]:
# Learning Rate Scheduleオブジェクト
# 学習率を変化させる場合に使用
# 0.0002から始まり、1000ステップごとに0.9倍になる
learning_rate = dc.models.optimizers.ExponentialDecay(0.0002, 0.9, 1000)
model = dc.models.MultitaskClassifier(
                                        n_tasks=len(tasks),
                                        n_features=1024,
                                        layer_size=[1000],
                                        dropouts=0.2,
                                        learning_rate=learning_rate
                                      )
model.fit(train_dataset, nb_epoch=50, callbacks=callback)


Step 1000 validation: roc_auc_score=0.748024
Step 2000 validation: roc_auc_score=0.769393
Step 3000 validation: roc_auc_score=0.760023
Step 4000 validation: roc_auc_score=0.776585
Step 5000 validation: roc_auc_score=0.765759
Step 6000 validation: roc_auc_score=0.760809
Step 7000 validation: roc_auc_score=0.764215
Step 8000 validation: roc_auc_score=0.767366
Step 9000 validation: roc_auc_score=0.772326
Step 10000 validation: roc_auc_score=0.775406
Step 11000 validation: roc_auc_score=0.76536
Step 12000 validation: roc_auc_score=0.765094
Step 13000 validation: roc_auc_score=0.768972
Step 14000 validation: roc_auc_score=0.766266
Step 15000 validation: roc_auc_score=0.765019
Step 16000 validation: roc_auc_score=0.765372


0.23679210662841796